<a href="https://colab.research.google.com/github/luistorres109/ciencia_dados_ia_trabalho_final/blob/main/reconhecimento_irregularidades/trabalho_final_luis_ds_ia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# AVISO LEGAL!!!
# Isto se trata de um trabalho acadêmico e não está sendo feita nenhuma acusação de fato se há alguma ilegalidade ou não nas licitações, e quem tem que decidir isso é o MP e a Justiça Brasileira
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/editais_dados.csv", delimiter=';')
print(df)

           Data Horario    Edital  \
0    2023-05-02     NaN  029/2023   
1    2023-05-10     NaN  135/2023   
2    2023-05-22     NaN  042/2023   
3    2023-05-22   16:00  138/2023   
4    2023-05-22   15:00  140/2023   
..          ...     ...       ...   
146  2023-08-23   16:00  297/2023   
147  2023-08-24   14:00  244/2023   
148  2023-08-28   10:10  301/2023   
149  2023-08-30   16:00  283/2023   
150  2023-09-05   08:40  294/2023   

                                                 Local  \
0                                                  NaN   
1                                                  NaN   
2                                                  NaN   
3                                         licitações-e   
4                                         licitações-e   
..                                                 ...   
146  Setor de Compras e Licitações na Prefeitura Mu...   
147  Setor de Compras e Licitações na Prefeitura Mu...   
148  Setor de Compras e Licitaçõe

In [ ]:
# Substituir vírgulas por pontos e converter a coluna "Montante" para valores float
df['Montante'] = df['Montante'].replace('\.', '', regex=True).replace(',', '.', regex=True).astype(float)

# Separar as características (features) e o alvo (target)
df_features = df.drop(["Montante", "Link da Página", "Horario", "Local"], axis=1)
df_target = df['Montante']

# Separar variáveis categóricas
categorical_cols = ['Data', 'Edital', 'Modalidade', 'Forma julgamento', 'Unidade gestora', 'Secretária', 'Objeto', 'Tipo']  # Substitua com suas colunas categóricas

# Inicializar o codificador de categorias
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Ajustar e transformar as variáveis categóricas
df_encoded = encoder.fit_transform(df_features[categorical_cols])

# Obter os nomes das colunas codificadas
encoded_columns = [f"{col}_{val}" for col, values in zip(categorical_cols, encoder.categories_) for val in values]

# Criado DF com as colunas codificadas
df_features_encoded = pd.DataFrame(df_encoded, columns=encoded_columns)

# Inicializar o modelo Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Treinar o modelo
rf_model.fit(df_features_encoded, df_target)

# Fazer previsões em todo o conjunto de dados
df['Montante_Pred'] = rf_model.predict(df_features_encoded)

# Calcular a diferença entre os valores reais e previstos
df['Desvio'] = df['Montante'] - df['Montante_Pred']

# Definição de um limiar para identificar desvios significativos
limiar_desvio = 1000000

# Filtrar licitações com desvios significativos
df = df[abs(df['Desvio']) > limiar_desvio].drop(['Data', 'Horario', 'Local', 'Modalidade', 'Forma julgamento', 'Unidade gestora', 'Secretária', 'Objeto', 'Tipo', 'Link da Página'], axis=1)
df = df.loc[df['Desvio'] >= 0]

# Visualizar as licitações irregulares
print("Dataframe com os editais (SUPOSTAMENTE) irregulares:")
print(df)
print(f"\nO numero total dessas licitacoes eh: {df[df.columns[0]].count()}")

Dataframe com os editais (SUPOSTAMENTE) irregulares:
       Edital     Montante  Montante_Pred        Desvio
2    042/2023  10556017.14   6.311167e+06  4.244850e+06
23   168/2023   3845378.86   2.378604e+06  1.466775e+06
30   022/2023   7428000.00   3.966299e+06  3.461701e+06
80   061/2023  93733018.68   5.819120e+07  3.554182e+07
117  154/2023   4500000.00   2.917920e+06  1.582080e+06
129  035/2023   3640545.00   2.384379e+06  1.256166e+06
149  283/2023   2762970.00   1.693218e+06  1.069752e+06

O numero total dessas licitacoes eh: 7
